<a href="https://colab.research.google.com/github/Rioto3/gbizinfo-subsidy-analysis/blob/main/gbizinfo_miyagi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gBizINFO APIを用いた補助金データ分析パイプライン

このノートブックでは、gBizINFO APIから宮城県の過去の補助金データを収集し、「既得権益の可視化」に向けた分析を実施します。特に補助金受給回数に着目した統計分析を通じて、農業分野における構造的な補助金依存の実態を明らかにすることが目的です。

**なおこのノートブックの目的と範囲は、データ分析において有意なCSVの作成までとします。**

[参考：gBizINFO API](https://info.gbiz.go.jp/api/index.html)


**最終的なテーブル**のイメージ
（このテーブルを作ることで、受領回数から傾向を考察できる状態を目指す）：

| 企業ID | 企業名  | 所在地 | 補助金受領回数 |
|--------|--------|--------|----------------|
| 3      | 企業C  | 宮城県 | 3              |
| 1      | 企業A  | 宮城県 | 2              |
| 2      | 企業B  | 宮城県 | 1              |
| 4      | 企業D  | 宮城県 | 1              |


## データソース

簡単なgBizINFO APIコードを示して、利用の可能性を明らかにする

In [ ]:
# APIの基本設定
API_TOKEN = "YOUR_API_TOKEN"  # 実際のAPIトークンに置き換えてください
BASE_URL = "https://info.gbiz.go.jp/hojin" # gBizINFO REST API エンドポイント
HEADERS={
    'accept': 'application/json',
    'X-hojinInfo-api-token': API_TOKEN
}

### GET /v1/hojin/{corporate_number}
gBizINFOに登録された法人の検索

In [ ]:
import requests
from pprint import pprint

response = requests.get(
    f"{BASE_URL}/v1/hojin", headers=HEADERS,
    params={
        'prefecture': '04', # 宮城県に限定
        'exist_flg': 'true', # 法人活動情報があるものに限定
        'source': '4', # 出典元で、補助金に関係するものに限定
        'limit': '1', # 1ページに含める件数
        'page': '1', # ページ番号を1ページ目に限定
    }
)

pprint(response.json())

### GET /v1/hojin/{corporate_number}/subsidy
特定の法人の補助金情報の検索

In [ ]:
corporate_number = "1000020042021" # 上記で得られた法人のIDを入力とします

response = requests.get(
  f'{BASE_URL}/v1/hojin/{corporate_number}/subsidy', headers=HEADERS,
)
pprint(response.json())

## データベース設計

上記のAPIの結果を保存するDBを用意
APIの特性上、補助金データの取得には法人IDが必要となること、から法人テーブルを用意する必要がある。
また、補助金データには法人名が直接記載がないため、法人IDを外部キーとして、法人データから外部結合する方針にする。

### テーブル定義

In [ ]:
from sqlalchemy import create_engine, Column, String, Integer, Date, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base

# データベースエンジンの作成（SQLiteを使用）
DATABASE_URL = 'sqlite:///./gbizinfo_miyagi.sqlite3'
engine = create_engine(DATABASE_URL, echo=True)
Base = declarative_base()

# 宮城県法人テーブルの定義
class MiyagiCompanies(Base):
    __tablename__ = 'companies'

    corporate_number = Column(String, primary_key=True)
    name = Column(String)
    location = Column(String)

# 宮城県補助金テーブルの定義
class MiyagiSubsidies(Base):
    __tablename__ = 'subsidies'

    id = Column(Integer, primary_key=True, autoincrement=True)
    corporate_number = Column(String, ForeignKey('companies.corporate_number'))
    subsidy_title = Column(String)
    amount = Column(Integer)
    date_of_approval = Column(Date)
    government_department = Column(String)
    target = Column(String)
    fiscal_year = Column(String)

    company = relationship('MiyagiCompanies', backref='subsidies')


## データ収集

### テーブルの作成

In [ ]:
Base.metadata.create_all(engine)

### 1.法人データ収集

In [ ]:
import requests
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import exists

# データベース設定
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

# データ取得＆DB保存
for page in range(1, 11):  # page=1 から page=10（APIの仕様の最大値） まで
    response = requests.get(
        f"{BASE_URL}/v1/hojin", headers=HEADERS,
        params={
            'prefecture': '04',
            'exist_flg': 'true',
            'source': '4',
            'page': page,
            'limit': 5000, # APIの仕様の最大値を設定し、最大数の結果取得を目指す
        }
    )

    if response.status_code != 200:
        print(f"エラー: {response.status_code}")
        continue

    data = response.json()

    for company in data.get('hojin-infos', []):
        # 企業の法人番号がすでにDBに存在するか確認
        exists_check = session.query(exists().where(MiyagiCompanies.corporate_number == company.get('corporate_number'))).scalar()

        if not exists_check:
            new_company = MiyagiCompanies(
                corporate_number=company.get('corporate_number'),
                name=company.get('name'),
                location=company.get('location')
            )
            session.add(new_company)
            print(f"新しい企業データを保存しました: {company.get('corporate_number')}")
        else:
            print(f"重複企業（法人番号: {company.get('corporate_number')}）はスキップしました")

    session.commit()  # 取得したデータをコミット
    print(f"Page {page} のデータを保存しました")

print("データ取得完了！")


### 2.補助金のデータ収集

In [ ]:
import requests
from datetime import datetime
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

# データベース接続
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()


def get_fiscal_year(date_str):
    """日付から年度を計算"""
    date = datetime.strptime(date_str, '%Y-%m-%d').date()
    return str(date.year) if date.month >= 4 else str(date.year - 1)


# 企業リストを取得
companies = session.query(MiyagiCompanies).all()


for company in companies:
    corporate_number = company.corporate_number
    response = requests.get(f'{BASE_URL}/v1/hojin/{corporate_number}/subsidy', headers=HEADERS)

    if response.status_code == 200:
        data = response.json()
        hojin_infos = data.get("hojin-infos", [])

        if hojin_infos:
            subsidies = hojin_infos[0].get("subsidy", [])

            for sub in subsidies:
                date_of_approval = sub.get("date_of_approval")

                if date_of_approval:
                    date = datetime.strptime(date_of_approval, '%Y-%m-%d').date()
                    fiscal_year = get_fiscal_year(date_of_approval)

                    subsidy_record = MiyagiSubsidies(
                        corporate_number=corporate_number,
                        subsidy_title=sub.get("title"),
                        amount=int(sub.get("amount") or 0),
                        date_of_approval=date,
                        government_department=sub.get("government_departments"),
                        target=sub.get("target"),
                        fiscal_year=fiscal_year
                    )
                    session.add(subsidy_record)  # データの追加を忘れずに
    else:
        print(f"Error fetching data for {corporate_number}: {response.status_code}")

session.commit()  # 重要: すべてのデータをコミット
session.close()
print("補助金データの取得＆保存完了！")


## データ品質検証

In [ ]:
import pandas as pd
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

# データベース接続
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

def query_to_dataframe(model):
    """SQLAlchemyのモデルをDataFrameに変換する"""
    df = pd.DataFrame([record.__dict__ for record in session.query(model).all()])
    df.drop(columns=['_sa_instance_state'], errors='ignore', inplace=True)
    return df

# 各テーブルのデータを取得してDataFrame化(セッションをリスタートした場合、データベース設計のセルを再実行してください。)
df_subsidies = query_to_dataframe(MiyagiSubsidies)
df_companies = query_to_dataframe(MiyagiCompanies)

session.close()


### 統計情報の確認

補助金データ基本情報

In [ ]:
df_companies.info()

In [ ]:
df_subsidies.info()

基本統計量

In [ ]:
df_subsidies.describe()

欠損値の数

In [ ]:
df_subsidies.isnull().sum()

重複レコード数

In [ ]:
df_companies.duplicated().sum()

In [ ]:
df_subsidies.duplicated().sum()

上位データのチェック

In [ ]:
df_companies.head()

In [ ]:
df_subsidies.head()

## データ分析

先に法人データ結合

In [ ]:
df_merge = df_subsidies.merge(df_companies, on='corporate_number', how='left')

補助金テーブル(法人名付き)の完成

In [ ]:
df_subsidies_with_name = df_merge[['corporate_number','name','subsidy_title','amount','date_of_approval']]
df_subsidies_with_name = df_subsidies_with_name.sort_values('corporate_number')
df_subsidies_with_name.to_csv("miyagi_subsidies.csv", index="false")
df_subsidies_with_name.head()

補助金の回数テーブルの完成

In [ ]:
df_subsidies_count =df_subsidies_with_name.groupby('corporate_number').agg(
    name=('name','first'),
    subsidy_count=('name', 'count'), # カウント
    total_amount=('amount', 'sum'),  # 合計金額を集計
).reset_index().rename(columns={'subsidy_count': 'subsidy_count', 'total_amount':'total_amount'})

df_subsidies_count = df_subsidies_count.sort_values(['subsidy_count', 'total_amount'], ascending=[False, False])
df_subsidies_count = df_subsidies_count.sort_values('total_amount', ascending=False)

df_subsidies_count.to_csv("miyagi_subsidies_count.csv", index="false")
df_subsidies_count.head()


簡単な可視化

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.scatterplot(x=range(len(df_subsidies_count)), y=df_subsidies_count["subsidy_count"], color="blue", alpha=0.7)

plt.xlabel("Company Index")
plt.ylabel("Subsidy Count")
plt.title("Distribution of Subsidy Count")

plt.grid(True, linestyle="--", alpha=0.5)

plt.show()

## データセット作成作業の終了：

### 得られたデータセット
- `miyagi_subsidies.csv`: 補助金データ（法人名付き）
- `miyagi_subsidies_count.csv`: 補助金受給回数・総額集計データ

### 次のステップ
1. 生成AIツールにこれらのCSVをインプット
2. 以下の観点から分析を依頼
   - 補助金受給回数に着眼した既得権益の構造の客観的分析
   - 補助金受給回数**以外**の着眼点の提供と分析

### 注意点
- 分析結果の解釈には批判的思考が必要